In [9]:
from google.cloud import bigquery
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.cloud import bigquery

In [15]:
isLocal=False

map_sevirity_to_class={'Cosmatic':0,'Minor': 1, "Major": 2, "Critical": 3}
print(f"Map severity Name to LabelCode: {str(map_sevirity_to_class)}")

# Write to a file
# BUCKET_URI = "gs://smart-ml-pongthorn"  
# LabelCode_FILE = "incident_sevirity_to_class.json"

# with open(LabelCode_FILE, "w") as outfile:
#     json.dump(map_sevirity_to_class, outfile)
# ! gsutil cp {LabelCode_FILE} {BUCKET_URI}

Map severity Name to LabelCode: {'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [16]:
if isLocal:
    print("Load ML Data from local")
    df=pd.read_csv("data/ML_Incident.csv")
else:
    print("Load ML Data from BQ")
    bqclient = bigquery.Client()
    table = bigquery.TableReference.from_string("pongthorn.SMartML.ml_incident")
    df=bqclient.list_rows(table).to_dataframe(create_bqstorage_client=False)

df.info()

Load ML Data from BQ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120 entries, 0 to 2119
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_id                2120 non-null   Int64  
 1   severity_name              2120 non-null   object 
 2   sla                        2120 non-null   object 
 3   product_type               2120 non-null   object 
 4   brand                      2120 non-null   object 
 5   service_type               2120 non-null   object 
 6   incident_type              2120 non-null   object 
 7   open_to_close_hour         2120 non-null   float64
 8   response_to_resolved_hour  2120 non-null   float64
dtypes: Int64(1), float64(2), object(6)
memory usage: 151.3+ KB


In [20]:
df['label_multi_severity'] =df['severity_name'].map(map_sevirity_to_class) 
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

print( list(df['label_multi_severity'].unique()))
print(df['label_binary_severity'].unique())
df.head()

[2, 1, 0, 3]
[1 0]


,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,label_multi_severity,label_binary_severity
0,2,Major,24x7 4Hrs Resolution Time,Other,F5,Request,General Incident,147.883333,147.666667,2,1
1,3,Minor,24x7 4Hrs Resolution Time,Other,F5,Incident,Software,168.550000,55.866667,1,0
2,3,Minor,24x7 4Hrs Resolution Time,Other,F5,Incident,General Incident,570.933333,361.666667,1,0
3,3,Minor,24x7 4Hrs Resolution Time,Other,F5,Incident,General Incident,306.716667,184.016667,1,0
4,3,Minor,24x7 4Hrs Resolution Time,Other,F5,Incident,General Incident,478.766667,478.516667,1,0


In [21]:
len_all=len(df)
print(f"All Data = {len_all}")

train, val = train_test_split(df,test_size=0.2,random_state=1000,shuffle=True)
val,test = train_test_split(val, test_size=0.5,random_state=1000,shuffle=True)

len_all=len(df)
len_train=len(train)
len_val=len(val)
len_test=len(test)
print(f'{len_train} =train examples ({round(len_train/len_all*100,1)}%)')
print(f'{len_val} =val examples ({round(len_val/len_all*100,1)}%)')
print(f'{len_test} =test examples ({round(len_test/len_all*100,1)}%)')

All Data = 2120
1696 =train examples (80.0%)
212 =val examples (10.0%)
212 =test examples (10.0%)


In [29]:
client = bigquery.Client()
def loadDataFrameToBQ(table_id,dfx):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            dfx, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print(f"{len(dfx)} rows imported to {table_id} successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))


In [30]:
if isLocal:
    train.to_csv("data/train_incident.csv",index=False)
    val.to_csv("data/validation_incident.csv",index=False)
    test.to_csv("data/test_incident.csv",index=False)
else:
    try:
     loadDataFrameToBQ("pongthorn.SMartML.train_incident",train)
     loadDataFrameToBQ("pongthorn.SMartML.validation_incident",val)
     loadDataFrameToBQ("pongthorn.SMartML.test_incident",test)   
    except Exception as ex:
     raise ex
    

1696 rows imported to pongthorn.SMartML.train_incident successfully
212 rows imported to pongthorn.SMartML.validation_incident successfully
212 rows imported to pongthorn.SMartML.test_incident successfully
